In [54]:
import numpy as np
import torch
import pandas as pd
import networkx as nx
import hypernetx as hnx
import itertools
from sklearn.cluster import KMeans
import tensorly as tl

In [55]:
# Generate a random tensor
n_nodes = 100
m = 3
K = 10
dim = [K] * m

# Randomly assign each node to a cluster
clusters = np.random.randint(0,K,n_nodes)

# One-hot encoding of the clusters
one_hot = np.zeros((n_nodes,K))
for i in range(n_nodes):
    one_hot[i,clusters[i]] = 1
one_hot = torch.tensor(one_hot)

P_hat = torch.zeros(dim)
for i in range(K):
    for j in range(i,K):
        for k in range(j,K):
            # Random variable uniformly distributed between 0 and 1
            p = np.random.uniform()
            P_hat[i,j,k] = p
            P_hat[i,k,j] = p
            P_hat[j,i,k] = p
            P_hat[j,k,i] = p
            P_hat[k,i,j] = p
            P_hat[k,j,i] = p

In [56]:
# Print the size of P_hat and one_hot
print(P_hat.size())
print(one_hot.shape)

torch.Size([10, 10, 10])
torch.Size([100, 10])


In [57]:
tl.set_backend('pytorch')
P_hat = P_hat.float()
one_hot = one_hot.float()
Q = tl.tucker_to_tensor((P_hat, [one_hot,one_hot,one_hot]))

In [58]:
Q.shape

torch.Size([100, 100, 100])

In [117]:
# The adjacency tensor must be symmetric, then the factor matrices are same, so we can use the first one
factor = factors[0]

# Apply scale-invariant to the factor matrix
factor = scale_invariant(factor)
R_hat = factor[:, 1:]

# Apply K-mean Clustering to the rows of factor matrix
kmeans = KMeans(n_clusters=K, random_state=0).fit(R_hat)
labels = kmeans.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [118]:
# Calculate the accuracy of the clustering
accuracy = np.sum(labels == clusters) / n_nodes
print(accuracy)

0.1


In [5]:
tl.set_backend('pytorch')

In [23]:
import torch

def higher_order_orthogonal_iteration(tensor, ranks, n_iter_max=1000, tol=1e-10):
    """
    Perform Higher Order Orthogonal Iteration (HOOI) on a 3D tensor.

    Parameters:
    - tensor: Input tensor of size I x J x K.
    - ranks: Tuple of target ranks (r1, r2, r3).
    - n_iter_max: Maximum number of iterations.
    - eps: Convergence criterion.

    Returns:
    - L, R, V factor matrices.
    - core_tensor: Core tensor of the Tucker decomposition.
    """
    # Initialize factor matrices
    I, J, K = tensor.shape
    L = torch.randn(I, ranks[0])
    R = torch.randn(J, ranks[1])
    V = torch.randn(K, ranks[2])

    # Initialize core tensor
    core_tensor = torch.zeros(ranks)

    # Iterate until convergence
    for _ in range(n_iter_max):
        # Update L
        L = tl.tenalg.multi_mode_dot(tensor, [R, V], modes=[1, 2], transpose=True)
        L = torch.linalg.qr(L)[0]

        # Update R
        R = tl.tenalg.multi_mode_dot(tensor, [L, V], modes=[0, 2], transpose=True)
        R = torch.linalg.qr(R)[0]

        # Update V
        V = tl.tenalg.multi_mode_dot(tensor, [L, R], modes=[0, 1], transpose=True)
        V = torch.linalg.qr(V)[0]

        # Update core tensor
        core_tensor = tl.tenalg.multi_mode_dot(tensor, [L, R, V], modes=[0, 1, 2])

        # Compute the approximation error
        error = torch.norm(tensor - core_tensor) / torch.norm(tensor)
        if error < tol:
            break

    return L, R, V, core_tensor

# Assuming we have a tensor 'A' and ranks r1, r2, r3 defined elsewhere.
# The actual tensor A and its dimensions should be defined here.
# For example:
# A = torch.randn(10, 10, 10)
# ranks = (3, 3, 3)
# L, R, V, core_tensor = higher_order_orthogonal_iteration(A, ranks)

# This code would then be executed with A and ranks defined, but for the purpose
# of this example, we won't execute it as we don't have a real tensor A to work with.


In [24]:
# Example tensor of size 10 x 10 x 10
A = torch.randn(10, 10, 10)
# Desired ranks for the decomposition
ranks = (3, 3, 3)
# Perform HOOI
L, R, V, core_tensor = higher_order_orthogonal_iteration(A, ranks)


TypeError: transpose() received an invalid combination of arguments - got (list), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)


In [39]:
R = torch.randn(4, 2)

In [45]:
# Check if R has orthogonal columns
R[:,0] @ R[:,1]


-2.037881221987743e-17

In [3]:
from scipy.stats import ortho_group
R = ortho_group.rvs(4)[:,:2]
R.shape

(4, 2)

In [1]:
import torch

tensor = torch.rand(6, 6, 6)
rank = (2, 2, 2)

In [4]:
I, J, K = tensor.shape
R = ortho_group.rvs(J)[:,:rank[1]]
R = torch.tensor(R, dtype=torch.float32)
V = ortho_group.rvs(K)[:,:rank[2]]
V = torch.tensor(V, dtype=torch.float32)

In [5]:
R.T.shape

torch.Size([2, 6])

In [6]:
import tensorflow as tf

def n_mode_product(x, u, n):
    n = int(n)
    # We need one letter per dimension
    # (maybe you could find a workaround for this limitation)
    if n > 26:
        raise ValueError('n is too large.')
    ind = ''.join(chr(ord('a') + i) for i in range(n))
    exp = f'{ind}K...,JK->{ind}J...'
    result = tf.einsum(exp, x, u)
    return torch.tensor(result.numpy())

In [17]:
def multi_mode_dot(tensor, matrices, modes):
    res = tensor
    for mode, matrix in zip(modes, matrices):
        res = n_mode_product(res, matrix, mode)
    return res

In [8]:
C = n_mode_product(tensor, R.T, 1)


In [21]:
C.shape

torch.Size([6, 2, 2])

In [10]:
type(C)

torch.Tensor

In [11]:
C = n_mode_product(C, V.T, 2)

In [20]:
C = multi_mode_dot(tensor,[R.T, V.T], [1, 2])

In [22]:
def unfold(tensor, mode):
    
    return torch.reshape(torch.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1))

In [23]:
C_1 = unfold(tensor, 0)

In [27]:
C_1.shape

torch.Size([6, 36])

In [29]:
U, _, _ = torch.svd(C_1)
L = U[:, :rank[0]]

In [30]:
L.shape

torch.Size([6, 2])

In [59]:
def HOOI(tensor, ranks, n_iter_max=100, tol=1e-8):
    """High-Order Orthogonal Iteration (HOOI) algorithm for Tucker decomposition
    Perform Higher Order Orthogonal Iteration (HOOI) on a 3D tensor.

    Parameters:
    - tensor: Input tensor of size I x J x K.
    - ranks: Tuple of target ranks (r1, r2, r3).
    - n_iter_max: Maximum number of iterations.
    - eps: Convergence criterion.

    Returns:
    - core_tensor: Core tensor of the Tucker decomposition.
    - [L, R, V] factor matrices.
    L ∈ R^I×r1, R ∈ R^J×r2, V ∈ R^K×r3.
    """
    # Initialize R and V factor matrices with orthonormal columns
    I, J, K = tensor.shape
    R = ortho_group.rvs(J)[:,:ranks[1]]
    R_hat = torch.tensor(R, dtype=torch.float32)
    V = ortho_group.rvs(K)[:,:ranks[2]]
    V_hat = torch.tensor(V, dtype=torch.float32)

    for _ in range(n_iter_max):
        # C = A ×2 R^T ×3 V^T
        c = multi_mode_dot(tensor, [R_hat.T, V_hat.T], modes=[1, 2])
        c_1 = unfold(c, 0)

        # L = SVD(r1, C_1), where U = SVD(k, C) means compute the k’th order truncated 
        # SVD of C and then set U = [u1, u2, . . . , uk] to the matrix whose columns are 
        # the k largest left singular vectors ui of C
        u, _, _ = torch.svd(c_1)
        L_hat = u[:, :ranks[0]]

        # D = A ×1 L^T ×3 V^T
        d = multi_mode_dot(tensor, [L_hat.T, V_hat.T], modes=[0, 2])
        d_2 = unfold(d, 1)

        #R = SVD(r2, D_2)
        u, _, _ = torch.svd(d_2)
        R_hat = u[:, :ranks[1]]

        # E = A ×1 L^T ×2 R^T
        e = multi_mode_dot(tensor, [L_hat.T, R_hat.T], modes=[0, 1])
        e_3 = unfold(e, 2)

        # V = SVD(r3, E_3)
        u, _, _ = torch.svd(e_3)
        V_hat = u[:, :ranks[2]]

        # Compute the approximation error
        core_tensor = n_mode_product(e, V_hat.T, 2)
        appr = multi_mode_dot(core_tensor, [L_hat, R_hat, V_hat], modes=[0, 1, 2])
        if torch.norm(tensor - appr) < tol:
            break

    return core_tensor, [L_hat, R_hat, V_hat]


In [50]:
core, factors = HOOI(tensor, rank)

In [51]:
appr = multi_mode_dot(core, factors, modes=[0, 1, 2])

In [52]:
torch.norm(tensor - appr)

tensor(4.2515)

In [60]:
# Apply the Tucker decomposition to find the factor matrices
core, factors = HOOI(Q, ranks=[K,K,K])

KeyboardInterrupt: 